In [ ]:
import os
import csv
import nltk
import re
import operator
import math
import readability
import random
from nltk import word_tokenize
from gensim.models import Word2Vec
from numpy import array
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from numpy import array
from numpy import mean
from numpy import zeros
from nltk.tag import StanfordPOSTagger
from nltk.tag import StanfordNERTagger

# java_path = "C:/Program Files/Java/jdk1.8.0_144/bin"
# os.environ['JAVAHOME'] = java_path

# nltk.internals.config_java("C:/Program Files/Java/jdk1.8.0_144/bin", options='-Xmx3024m')


st = StanfordPOSTagger('english-left3words-distsim.tagger', path_to_jar='stanford-postagger.jar')
ner = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz', 'stanford-ner.jar')

def train_test_split(directory):
    file_names = os.listdir(directory)
    random.shuffle(file_names)
    train_list = []
    test_list = []
    i = 0
    for file_name in file_names:
        if file_name != '.DS_Store':
            if i < 4:
                train_list.append(file_name)
                i+= 1
            else:
                test_list.append(file_name)
                i = 0
    return (train_list, test_list)

def make_training_dicts(filepath, file_list):
    age_dir = {}
    time_dir = {}
    with open(filepath, encoding='latin-1') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[0] in file_list:
                age_dir[row[0]] = row[1]
                time_dir[row[0]] = row[2]
    return (age_dir, time_dir)

def clean_and_tokenize_file(file_name):
    file = open(file_name, encoding='latin-1')
    raw_text = file.read()
    raw_text_lower = raw_text.lower()
    caseless_tokens = nltk.word_tokenize(raw_text_lower)
    caseless_tokens = [x for x in caseless_tokens if re.match('\w+', x)]
    sentences = nltk.sent_tokenize(raw_text_lower)
    raw_sentence_tokens = nltk.sent_tokenize(raw_text)
    case_tokens = nltk.word_tokenize(raw_text)
    sentence_list = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        sentence_list.append([x for x in words if re.match('\w+', x)])
    return (caseless_tokens, sentence_list, raw_sentence_tokens, case_tokens)


def clean_and_tokenize_corpus(directory, file_list):
    file_dir = {}
    for file_name in os.listdir(directory):
        if file_name in file_list:
            file_dir[file_name] = clean_and_tokenize_file(directory + file_name)
    return file_dir


def count_total_vocab(file_wordfreqs):
    vocab = {}
    for words in file_wordfreqs:
        for word in words:
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    return vocab

def get_corpus_vocab(corpus, unknown_threshold):
    initial_dict = count_total_vocab(corpus)
    sorted_dict = sorted(initial_dict.items(), key=operator.itemgetter(1), reverse=True)
    filtered_dict = [key for key, value in sorted_dict[:unknown_threshold]]
    return filtered_dict

def get_idf_dict(vocab, corpus):
    idf_dict = {}
    num_docs = len(corpus) 
    for word in vocab:
        df_count = 1
        for doc in corpus:
            if word in doc:
                df_count += 1
        idf_dict[word] = math.log(num_docs / df_count)
    return idf_dict

def get_named_entities(novel):
    named_entities = ner.tag(novel)
    entities = []
    cur_entity = []
    last_tag = 'O'
    for entity in named_entities:
        if entity[1] == 'O':
            if len(cur_entity) > 0:
                new_entry = (' '.join(cur_entity), last_tag)
                if new_entry not in entities:
                    entities.append(new_entry)
                cur_entity = []
            last_tag = 'O'
        elif entity[1] == last_tag:
            cur_entity.append(entity[0])
        else:
            if len(cur_entity) > 0:
                new_entry = (' '.join(cur_entity), last_tag)
                if new_entry not in entities:
                    entities.append(new_entry)
                cur_entity = []
            cur_entity.append(entity[0])
            last_tag = entity[1]
    return entities

def get_ne_corpus(corpus):
    ne_dict = {}
    for key, value in corpus.items():
        ne_dict[key] = get_named_entities(value[3])
    return ne_dict

full_list = train_test_split('chunk_50/clean')
train_list = full_list[0]
test_list = full_list[1]
training_data = make_training_dicts('chunk_50/clean.csv', train_list)
training_age = training_data[0]
training_time = training_data[1]
corpus = clean_and_tokenize_corpus('chunk_50/clean/', train_list)
ugram_corpus = [value[0] for key, value in corpus.items()]
vocab = get_corpus_vocab(ugram_corpus, 500)
idf_dict = get_idf_dict(vocab, ugram_corpus)
ne_corpus = get_ne_corpus(corpus)
ne_vals = [value for key, value in ne_corpus.items()]
ne_vocab = get_corpus_vocab(ne_vals, 100)
ne_idf_dict = get_idf_dict(ne_vocab, ne_vals)


testing_data = make_training_dicts('chunk_50/clean.csv', test_list)
testing_age = testing_data[0]
testing_time = testing_data[1]
test_corpus = clean_and_tokenize_corpus('chunk_50/clean/', test_list)
test_ugram_corpus = [value[0] for key, value in test_corpus.items()]

test_idf_dict = get_idf_dict(vocab, test_ugram_corpus)
test_ne_corpus = get_ne_corpus(test_corpus)
test_ne_vals = [value for key, value in test_ne_corpus.items()]
test_ne_idf_dict = get_idf_dict(ne_vocab, test_ne_vals)
print('Finished corpus-wide work')

In [ ]:
def train_word2vec(corpus):
    model = Word2Vec(corpus, size=100)
    model.train(corpus, total_examples=len(corpus), epochs=50)
    return model

word2vec = train_word2vec(ugram_corpus)
print('Finished word2vec training')

In [ ]:
import operator
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from gensim import corpora
from gensim.models.ldamodel import LdaModel

stop_words = set(stopwords.words('english')) 

def lemmatize_text(text):
    lemmatized = [WordNetLemmatizer().lemmatize(word) for word in text if word not in stop_words]
    return lemmatized

def lemmatize_corpus(corpus):
    clean_texts = [lemmatize_text(text) for text in corpus]
    return clean_texts

def get_working_dict(corpus):
    return corpora.Dictionary(corpus)
    
def train_lda(lem_corpus, dictionary):
    corpus = [dictionary.doc2bow(text) for text in lem_corpus]
    lda_model = LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
    return lda_model

lemmatized_corpus = lemmatize_corpus(ugram_corpus)
dictionary = get_working_dict(lemmatized_corpus)
lda_model = train_lda(lemmatized_corpus, dictionary)
print('Finished topic-model training')

In [ ]:
def create_name_feature_vector(is_time):
    tagset = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
    topic_list = []
    for i in range(10):
        topic_list.append('topic ' + str(i))
    init_list = vocab + tagset + ['w2v'] * 100 + topic_list
    init_list.append('vocab size')
    init_list.append('cs similarity')
    if is_time:
        time_list = init_list + ne_vocab
        time_list.append('flesch score')
        return time_list
    else:
        return init_list
    
time_feature_names = create_name_feature_vector(True)
age_feature_names = create_name_feature_vector(False)
print('Finished init vector names')
    

In [ ]:
st = StanfordPOSTagger('english-left3words-distsim.tagger', path_to_jar='stanford-postagger.jar')
st.java_options = '-mx4g'
def clean_and_tokenize_file(file_name):
    file = open(file_name)
    raw_text = file.read()
    raw_text_lower = raw_text.lower()
    caseless_tokens = nltk.word_tokenize(raw_text_lower)
    sentences = nltk.sent_tokenize(raw_text_lower)
    raw_sentence_tokens = nltk.sent_tokenize(raw_text)
    case_tokens = nltk.word_tokenize(raw_text)
    sentence_list = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        sentence_list.append([x for x in words if re.match('\w+', x)])
    return (caseless_tokens, sentence_list, raw_sentence_tokens, case_tokens)

def tf_idf(vocab, document, idf_dict):
    counts = Counter(document)
    doc_len = len(document)
    tf_idfs = []
    for word in vocab:
        doc_counter = 0
        tf = counts[word] / len(document)
        idf = idf_dict[word]
        tf_idfs.append(tf*idf)
    return tf_idfs

def get_avg_word2vec(novel):
    vec_list = []
    for word in novel:
        if word in word2vec:
            vec_list.append(word2vec[word])
    vector_array = array(vec_list)
    vector_array = mean(vector_array, axis=0)
    return vector_array.tolist()

def get_pairwise_similarity(sentences):
    sentence_vectors = []
    for sentence in sentences:
        vec_list = []
        for word in sentence:
            if word in word2vec:
                vec_list.append(word2vec[word])
        if len(vec_list) > 0:
            vector_array = array(vec_list)
            vector_array = mean(vector_array, axis=0)
            sentence_vectors.append(vector_array)
    cs_sim = cosine_similarity(sentence_vectors, sentence_vectors)
    return mean(cs_sim)

def get_pos_vector(novel, pos_tags):
    tagged_tokens = st.tag(novel)
    tags = [tag for (word, tag) in tagged_tokens]
    num_tokens = len(novel)
    pos_vector = [tags.count(pos_tag) / num_tokens for pos_tag in pos_tags]
    return pos_vector

# gets topic ids of top-ten topics
def get_topic_vector(novel):
    lem_novel = lemmatize_text(novel)
    new_text = dictionary.doc2bow(novel)
    suggested_topics = dict(lda_model.get_document_topics(new_text))
    topics = [suggested_topics[i] if i in suggested_topics else 0 for i in range(10)]
    return topics

def get_readability_score(sentences):
    newline_regex = re.compile(r'\n')
    clean_sents = []
    for sentence in sentences:
        clean_sents.append(re.sub(newline_regex, ' ', sentence))
    readability_text = '\n'.join(clean_sents)
    readability_results = readability.getmeasures(readability_text, lang='en')
    flesch_score = readability_results['readability grades']['FleschReadingEase']   
    return flesch_score

def feature_vector(novel, vocab, idf_dict, pos_tags):
    tokens = novel[0] # for unigrams
    token_sents = novel[1] #  for sent similarity
    case_tokens = novel[3] # for pos tagging and ner
    
    unigram_vector = tf_idf(vocab, tokens, idf_dict)
    pos_vector = get_pos_vector(case_tokens, pos_tags)
    word_2_vec_vector = get_avg_word2vec(tokens)
    topic_vector = get_topic_vector(tokens)
    word_vector = unigram_vector + pos_vector + word_2_vec_vector + topic_vector
    
    # gets ratio of unique words to total # of words
    novel_vocab = count_total_vocab([tokens])
    vocab_size = len(novel_vocab) / len(tokens)
    word_vector.append(vocab_size)
      
    cs_similarity = get_pairwise_similarity(token_sents)
    word_vector.append(cs_similarity)
    return word_vector

def feature_vector_time_period(age_vector, novel, ne_vocab, ne_novel, ne_idf_dict):
    raw_sents = novel[2]
    ne_vector = tf_idf(ne_vocab, ne_novel, ne_idf_dict)
    time_period_vector = age_vector.tolist() + ne_vector
    time_period_vector.append(get_readability_score(raw_sents))
    return time_period_vector

def create_age_vector_arrays(training_data, corpus, vocab, idf_dict, pos_tags):
    len_feature_vector = len(vocab) + len(pos_tags) + 112
    vector_array = zeros((len(training_data), len_feature_vector))
    results_array = zeros(len(training_data))
    index = 0
    for data, age in training_data.items():
        results_array[index] = age
        novel = corpus[data]
        vector_array[index] = feature_vector(novel, vocab, idf_dict, pos_tags)
        index += 1
    return (vector_array, results_array)

def create_time_vector_arrays(training_data, age_arrays, corpus, ner_vocab, ner_corpus, ner_idf_dict):
    len_feature_vector = len(age_arrays[0]) + len(ner_vocab) + 1
    vector_array = zeros((len(training_data), len_feature_vector))
    results_array = zeros(len(training_data))
    index = 0
    for data, time in training_data.items():
        results_array[index] = time
        novel = corpus[data]
        age_vector = age_arrays[index]
        vector_array[index] = feature_vector_time_period(age_vector, novel, ner_vocab, ner_corpus[data], ner_idf_dict)
        index += 1
    return (vector_array, results_array)

tagset = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB']
training_data_age = create_age_vector_arrays(training_age, corpus, vocab, idf_dict, tagset)
training_data_time = create_time_vector_arrays(training_time, training_data_age[0], corpus, ne_vocab, ne_corpus, ne_idf_dict)
testing_data_age = create_age_vector_arrays(testing_age, test_corpus, vocab, test_idf_dict, tagset)
testing_data_time = create_time_vector_arrays(testing_time, testing_data_age[0], test_corpus, ne_vocab, test_ne_corpus, test_ne_idf_dict)
print('finished feature arrays')

In [ ]:
from numpy import savetxt
savetxt("train_features_age.csv", training_data_age[0], delimiter=",")
savetxt("train_target_age.csv", training_data_age[1], delimiter=",")
savetxt("test_features_age.csv", testing_data_age[0], delimiter=",")
savetxt("test_target_age.csv", testing_data_age[1], delimiter=",")

savetxt("train_features_time.csv", training_data_time[0], delimiter=",")
savetxt("train_target_time.csv", training_data_time[1], delimiter=",")
savetxt("test_features_time.csv", testing_data_time[0], delimiter=",")
savetxt("test_target_time.csv", testing_data_time[1], delimiter=",")

In [ ]:
from math import floor
from statistics import mean
from numpy import array
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
# Function to scale input vectors
def scale(scaler, input_vector):
    scaler.fit(input_vector)
    scaled_features = scaler.transform(input_vector)
    return scaled_features
# Function to calculate number of nodes in hidden layer of MLP
def calc_node_num(input_len, output_len):
    return floor(mean([input_len, output_len]))
# Function to construct params dictionary to pass to cross validation
def construct_params(node_num):
    return {'n_estimators': [10, 50, 100], 
            'max_depth': [None, 5, 10],
            'max_features': ['auto', 'log2']}
# Function to run 5-fold cross validation (with MLP) to determine optimal hyperparameters for MLP classifer
def cross_validation(x_features, y_labels, params):
    clf = GridSearchCV(RandomForestClassifier(), params, cv=5,
                       scoring='accuracy') # Determine why F1 score was not working earlier
    clf.fit(x_features, y_labels)
    return clf.best_params_
# Return model with given params
def get_model(params):
    return RandomForestClassifier(n_estimators=params['n_estimators'], 
                         max_depth=params['max_depth'], 
                         max_features=params['max_features'])
# drop out unimportant features
def drop_out(model, x_vector, y_vector):
    selected_features_model = SelectFromModel(model)
    selected_features_model.fit(x_vector, y_vector)
    return selected_features_model

# Train model on entire training set
def train_model(model, x_vector, y_vector):
    model.fit(x_vector, y_vector)
    return model
# Steps for age and time period classifiers
# (0) Scale input/output data (?)
scaler = StandardScaler()
age_features = training_data_age[0]
age_labels = training_data_age[1]
scaled_age_features = scale(scaler, age_features)
# print(scaled_age_features)
time_period_features = training_data_time[0]
time_period_labels = training_data_time[1]
scaled_time_period_features = scale(scaler, time_period_features)
# print(scaled_time_period_features)
# (1) Perform cross validation with all chosen parameters
age_node_num = calc_node_num(len(age_features[0]), len(age_labels))
age_params = construct_params(age_node_num)
best_age_params = cross_validation(scaled_age_features, age_labels, age_params)
tp_node_num = calc_node_num(len(time_period_features[0]), len(time_period_labels))
time_period_params = construct_params(tp_node_num)
best_tp_params = cross_validation(scaled_time_period_features, time_period_labels, time_period_params)
# (2) After determining optimal params based on accuracy, retrain model with those params
age_model = get_model(best_age_params)
undropped_age_model = train_model(age_model, scaled_age_features, age_labels)
dropped_age_model = drop_out(undropped_age_model, scaled_age_features, age_labels)
important_age_features = dropped_age_model.transform(scaled_age_features)
trained_age_model = train_model(age_model, important_age_features, age_labels)

tp_model = get_model(best_tp_params)
undropped_tp_model = train_model(tp_model, scaled_time_period_features, time_period_labels)
dropped_tp_model = drop_out(undropped_tp_model, scaled_time_period_features, time_period_labels)
important_tp_features = dropped_tp_model.transform(scaled_time_period_features)
trained_tp_model = train_model(tp_model, important_tp_features, time_period_labels)

In [ ]:
def adjust_feature_labels(labels, sf_model):
    new_labels = []
    indexes = sf_model.get_support(indices=True)
    print(indexes)
    print(len(labels))
    new_labels = [labels[index] for index in indexes]
    return new_labels

age_feature_names = adjust_feature_labels(age_feature_names, dropped_age_model)
time_feature_names = adjust_feature_labels(time_feature_names, dropped_tp_model)

In [ ]:
# from math import floor
# from statistics import mean
# from numpy import array
# from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import classification_report
# from sklearn.model_selection import GridSearchCV

# # Function to scale input vectors
# def scale(scaler, input_vector):
#     scaler.fit(input_vector)
#     scaled_features = scaler.transform(input_vector)
#     return scaled_features

# # Function to calculate number of nodes in hidden layer of MLP
# def calc_node_num(input_len, output_len):
#     return floor(mean([input_len, output_len]))

# # Function to construct params dictionary to pass to cross validation
# def construct_params(node_num):
#     return {'hidden_layer_sizes': [(node_num,)], 
#             'activation': ['logistic', 'tanh', 'relu'],
#             'solver': ['lbfgs', 'sgd']}

# # Function to run 5-fold cross validation (with MLP) to determine optimal hyperparameters for MLP classifer
# def cross_validation(x_features, y_labels, params):
#     clf = GridSearchCV(MLPClassifier(), params, cv=5,
#                        scoring='accuracy') # Determine why F1 score was not working earlier
#     clf.fit(x_features, y_labels)
#     return clf.best_params_

# # Return model with given params
# def get_model(params):
#     return MLPClassifier(hidden_layer_sizes=params['hidden_layer_sizes'], 
#                          activation=params['activation'], 
#                          solver=params['solver'], max_iter=500)

# # Train model on entire training set
# def train_model(model, x_vector, y_vector):
#     model.fit(x_vector, y_vector)
#     return model

# # Steps for age and time period classifiers
# # (0) Scale input/output data (?)
# scaler = StandardScaler()
# age_features = training_data_age[0]
# age_labels = training_data_age[1]
# scaled_age_features = scale(scaler, age_features)
# # print(scaled_age_features)

# time_period_features = training_data_time[0]
# time_period_labels = training_data_time[1]
# scaled_time_period_features = scale(scaler, time_period_features)
# # print(scaled_time_period_features)

# # (1) Perform cross validation with all chosen parameters
# age_node_num = calc_node_num(len(age_features[0]), len(age_labels))
# age_params = construct_params(age_node_num)
# best_age_params = cross_validation(scaled_age_features, age_labels, age_params)

# tp_node_num = calc_node_num(len(time_period_features[0]), len(time_period_labels))
# time_period_params = construct_params(tp_node_num)
# best_tp_params = cross_validation(scaled_time_period_features, time_period_labels, time_period_params)

# # (2) After determining optimal params based on accuracy, retrain model with those params
# age_model = get_model(best_age_params)
# trained_age_model = train_model(age_model, scaled_age_features, age_labels)
# tp_model = get_model(best_tp_params)
# trained_tp_model = train_model(tp_model, scaled_time_period_features, time_period_labels)

In [ ]:
from random import randint
from sklearn.dummy import DummyClassifier
from yellowbrick.classifier import ClassPredictionError, ClassificationReport
from yellowbrick.features.importances import FeatureImportances
from numpy import append
import matplotlib.pyplot as plt
from yellowbrick.style import set_palette
    
# Palette only needs to be set globally once
# set_palette('set1') 

# Function to produce visualization of class prediction
def predict_class_error(model, training_features, training_labels, test_features, test_labels, class_labels):
    fig = plt.figure()
    ax = fig.add_subplot(111)
#     ax.set_xlim(auto=True)
    plt.setp(ax.get_xticklabels(), rotation=30)
    visualizer = ClassPredictionError(model, classes=class_labels, ax=ax)
    print("Accuracy: %s" % visualizer.score(test_features, test_labels))
    g = visualizer.poof()
    
# Function to produce visualization of metrics
def predict_metrics(model, training_features, training_labels, test_features, test_labels, class_labels):
    fig = plt.figure()
    ax = fig.add_subplot()
    visualizer = ClassificationReport(model, classes=class_labels, support=True, ax=ax)
    print("Accuracy: %s" % visualizer.score(test_features, test_labels))
    g = visualizer.poof()
    
# Function to produce visualization of most important features in classifying novel
# Note: new model refit with top 10 values of importance from entire model
def predict_feature_importance(model, best_params, features, targets, feature_names):
    importances = model.feature_importances_
    importance_name_dict = {}
    feature_index_dict = {}
    
    # Initialize importance to name mappings and feature name to index mappings
    for i in range(0, len(feature_names)):
        importance_name_dict[importances[i]] = feature_names[i]
        feature_index_dict[feature_names[i]] = i
    
    # Sort list of importances to find ten with highest importances
    highest_importances = (sorted(importances, reverse = True))[0:10]
    highest_names = []
    highest_indexes = []
    
    # For every highest importance, find matching name in related map
    for importance in highest_importances:
        name = importance_name_dict[importance]
        highest_names.append(name)
        highest_indexes.append(feature_index_dict[name])
        
    # Remove all features except those with highest importance
    new_features = []
    for feature in features:
        new_feature = [feature[i] for i in highest_indexes]
        new_features.append(new_feature)
    
    # Fit new model with extracted features to highlight their importance
    new_model = get_model(best_age_params)
    fig = plt.figure()
    ax = fig.add_subplot()
    visualizer = FeatureImportances(new_model, ax=ax, labels=highest_names)
    visualizer.fit(new_features, targets)
    g = visualizer.poof()
    
# (3) Use test set to return precision, recall, and f1 scores
print("AGE CALCULATIONS")
age_classes = ["18-24", "25-34", "35-49", "50-64", "65-x"]
test_age_features = testing_data_age[0]
test_age_labels = testing_data_age[1]
undropped_test_age_features = scale(scaler, test_age_features)
scaled_test_age_features = dropped_age_model.transform(undropped_test_age_features)

# Age predictions based on random assignment
print("RANDOM CALCULATIONS")
dummy_age_classifier = DummyClassifier()
trained_dummy_age_classifier = train_model(dummy_age_classifier, scaled_age_features, age_labels)
# predict_class_error(trained_dummy_age_classifier, scaled_age_features, age_labels, 
#                     scaled_test_age_features, test_age_labels, age_classes)
predict_metrics(trained_dummy_age_classifier, scaled_age_features, age_labels, 
                    scaled_test_age_features, test_age_labels, age_classes)

# Age Predictions based on trained model
print("FINAL CALCULATIONS")
# predict_class_error(trained_age_model, scaled_age_features, age_labels, 
#                     scaled_test_age_features, test_age_labels, age_classes)
print(trained_age_model)
predict_metrics(trained_age_model, scaled_age_features, age_labels, 
                    scaled_test_age_features, test_age_labels, age_classes)
predict_feature_importance(trained_age_model, best_age_params, scaled_age_features, age_labels, age_feature_names)

print("TIME PERIOD CALCULATIONS")
tp_classes = ["1751-1800", "1801-1820", "1821-1840", "1841-1860", "1861-1880", "1881-1900"]
test_tp_features = testing_data_time[0]
test_tp_features = append(test_tp_features, [test_tp_features[0]], axis=0)
test_tp_labels = testing_data_time[1]
test_tp_labels = append(test_tp_labels, 0)
undropped_test_tp_features = scale(scaler, test_tp_features)
scaled_test_tp_features = dropped_tp_model.transform(undropped_test_tp_features)

print("RANDOM CALCULATIONS")
dummy_tp_classifier = DummyClassifier()
trained_dummy_tp_classifier = train_model(dummy_tp_classifier, scaled_time_period_features, time_period_labels)
# predict_class_error(trained_dummy_tp_classifier, scaled_time_period_features, time_period_labels, 
#                    scaled_test_tp_features, test_tp_labels, tp_classes)
predict_metrics(trained_dummy_tp_classifier, scaled_time_period_features, time_period_labels, 
                    scaled_test_tp_features, test_tp_labels, tp_classes)

print("FINAL CALCULATIONS")
# predict_class_error(trained_tp_model, scaled_time_period_features, time_period_labels, 
#                    scaled_test_tp_features, test_tp_labels, tp_classes)
predict_metrics(trained_tp_model, scaled_time_period_features, time_period_labels, 
                    scaled_test_tp_features, test_tp_labels, tp_classes)
predict_feature_importance(trained_tp_model, best_tp_params, scaled_time_period_features, 
                           time_period_labels, time_feature_names)